In [ ]:
# ITCS 6150
# Noah Foster
# Project 2

In [62]:
import random
# helper function makes a 2d array size nxn filled with zeroes
def makeArray(n):
    array = []
    for i in range(0,n):
        inner = []
        for j in range(0, n):
            inner.append(0)
        array.append(inner)
    return array
# helper function generates n tuples (i,j) representing queens at position row i, column j with one in each row
def makeQueens(n):
    queens = []
    for i in range(0,n):
        queen = (i, random.randint(0,n-1))
        queens.append(queen)  
    return queens
# helper function to find queens attacking each square and populate array accordingly.
def populateAttacking(array, queens):
    for each in queens:
        (i,j)=each
        incrementRow(array, i)
        incrementColumn(array, j)
        incrementDiags(array, i, j)
        array[i][j]=array[i][j]-2 # decrement for repeated counts on the space we are sitting on.
# helper for populateAttacking to increment for attacks on same row
def incrementRow(array, i):
    j=0
    while j<len(array[i]):
        array[i][j]=array[i][j]+1
        j=j+1
# helper for populateAttacking to increment for attacks on same column
def incrementColumn(array, j):
    i=0
    while i<len(array):
        array[i][j]=array[i][j]+1
        i=i+1
# helper for populateAttacking to increment for attacks on the diagonal
def incrementDiags(array, qi, qj):
    i=0
    j=0
    while i<len(array):
        j=0
        while j<len(array[i]):
            if ((i+j == qi+qj) or (i-j==qi-qj)):
                array[i][j]=array[i][j]+1
            j=j+1
        i=i+1
# helper for determining if a square is attacked by a queen 
# (helpful because if it is, we want to consider that square to have a lower value when considering to move the queen there)
def isAttacking(queen,i,j):
    (qi,qj)=queen
    if (qi==i or qj==j or qi+qj==i+j or qi-qj==i-j):
        return 1
    else:
        return 0
        

# helper to show board of queens
def showQueens(n, queens):
    #n = len(queens)
    array = []
    for i in range(0,n):
        inner = []
        for j in range(0, n):
            inner.append(0)
        array.append(inner)
    for each in queens:
        (i,j) = each
        array[i][j] = 'Q'
    printArray(array)
        
    

# Helper Function to Print 2D Arrays.
def printArray(array):
    for row in array:
        string = ""
        for each in row:
            string = string + str(each) + " "
        print(string)
        
# Helper function to copy a 2d Array
def copy2dArray(array):
    array2 = []
    for each in array:
        array2.append(each.copy())
    return array2

# Helper Function Swaps Two Spaces In 2D array, returning the new array.
def swapSpaces(array,i1,j1,i2,j2):
    array2=copy2dArray(array)
    temp = array2[i2][j2]
    array2[i2][j2]=array2[i1][j1]
    array2[i1][j1]=temp
    return array2






In [534]:
# helper to find the best move for a given queen and array and array of queens
def bestMove(array,qi,qj,queens):
    besti=None
    bestj=None
    bestH=len(array)**2 # impossibly large so the first better value is always chosen
    qH = array[qi][qj]-1
    i=0
    while i<len(array):
        j=0
        while j<len(array[i]):
            h = array[i][j]
            if(isAttacking((qi,qj),i,j)): # lower h value because we don't want to consider attacks by the queen we are moving.
                h=h-1
            
            isOccupied=False
            for each in queens:
                (oi,oj)=each
                if (i==oi and j==oj):
                    isOccupied=True
            if(isOccupied): # we don't consider the square any queen is on as a possible move
                pass
            else: 
                if(h>=qH): # the current position of the queen is better
                    pass
                else:
                    if (h<bestH): # if, after all that, the h is less than the best h, we replace the best values
                        bestH=h
                        besti=i
                        bestj=j
            j=j+1
        i=i+1
    return(bestH,besti,bestj)

# helper to find sidewaysMoves
def sidewaysFinder(array,qi,qj,queens):
    qH = array[qi][qj]-1
    sideways=[]
    i=0
    for i in range(0,len(queens)):
        (testi,testj)=queens[i]
        if (testi==qi and testj==qj):
            q=i
    i=0
    while i<len(array):
        j=0
        while j<len(array[i]):
            h = array[i][j]
            if(isAttacking((qi,qj),i,j)): # lower h value because we don't want to consider attacks by the queen we are moving.
                h=h-1
            
            isOccupied=False
            for each in queens:
                (oi,oj)=each
                if (i==oi and j==oj):
                    isOccupied=True
            if(isOccupied): # we don't consider the square any queen is on as a possible move
                pass
            else: 
                if(h==qH): # sideways move found
                    sideways.append((q,i,j))
            j=j+1
        i=i+1
    return sideways
    

In [563]:
import heapq
# hill climb search
def hillClimbSearch(n, queens=None):
    steps=0
    if queens == None:
        queens = makeQueens(n)
    while (True):
        array = makeArray(n) 
        populateAttacking(array, queens) # populating the array with heuristic values
        priorityQueens = []
        heapq.heapify(priorityQueens) # array to store indexes of queens as sorted list by which one is worst
        q=0
        while(q<len(queens)): # sort queens by how many queens are attacking them
            (qi,qj)=queens[q]
            qH = array[qi][qj]-1
            heapq.heappush(priorityQueens, (qH*-1,q)) # heaps sort from smallest to largest and we want largest to smallest, so multiply by -1
            q=q+1
        (worstH,worst)=priorityQueens[0]
        if(worstH==0): # the worst queen is 0, we solved it
            return (queens, steps)
        qi=None
        qj=None
        qH=None
        (bestH,besti,bestj)=(None,None,None)
        while(len(priorityQueens)>0 and besti==None): # we want to check all the queens, stopping when we have an answer
            (qHNeg,q)=heapq.heappop(priorityQueens)
            qH=qHNeg*-1 # reverse the negativeness that we applied earlier
            (qi,qj)=queens[q] # position of the queen we are checking
            (bestH,besti,bestj)=bestMove(array,qi,qj,queens)
        if (besti==None): # the algorithm failed.
            return (None, steps)
        queens[q]=(besti,bestj)
        steps=steps+1
        

# hill climb search but it supports sideways moves up to a certain count in a row.
def hillClimbSearchSideways(n, maxSidewaysMoves, queens=None):
    steps=0
    if queens==None:
        queens = makeQueens(n)
    sidewaysMovesCount=0
    while (True):
        goodMoveMade=False
        array = makeArray(n) 
        populateAttacking(array, queens) # populating the array with heuristic values
        priorityQueens = []
        heapq.heapify(priorityQueens) # array to store indexes of queens as sorted list by which one is worst
        q=0
        while(q<len(queens)): # sort queens by how many queens are attacking them
            (qi,qj)=queens[q]
            qH = array[qi][qj]-1
            heapq.heappush(priorityQueens, (qH*-1,q)) # heaps sort from smallest to largest and we want largest to smallest, so multiply by -1
            q=q+1
        (worstH,worst)=priorityQueens[0]
        priorityQueensSideways=priorityQueens.copy()
        if(worstH==0): # the worst queen is 0, we solved it
            return (queens, steps)
        qi=None
        qj=None
        qH=None
        (bestH,besti,bestj)=(None,None,None)
        while(len(priorityQueens)>0 and besti==None): # we want to check all the queens, stopping when we have an answer
            (qHNeg,q)=heapq.heappop(priorityQueens)
            qH=qHNeg*-1 # reverse the negativeness that we applied earlier
            (qi,qj)=queens[q] # position of the queen we are checking
            (bestH,besti,bestj)=bestMove(array,qi,qj,queens)
        if (besti!=None): # a best move was found, proceed.
            queens[q]=(besti,bestj)
            #print("good move "+str(bestH))
            goodMoveMade=True
            sidewaysMovesCount=0
            steps=steps+1
        if (goodMoveMade==False):
            # by virtue of getting here, we have reached the sideways moves.
            if(sidewaysMovesCount<maxSidewaysMoves):
                qi=None
                qj=None
                qH=None
                (besti,bestj)=(None,None)
                while(len(priorityQueensSideways)>0 and besti==None): # we want to check all the queens, stopping when we have an answer
                    (qHNeg,q)=heapq.heappop(priorityQueensSideways)
                    qH=qHNeg*-1 # reverse the negativeness that we applied earlier
                    (qi,qj)=queens[q] # position of the queen we are checking
                    sidewaysMoves=sidewaysFinder(array,qi,qj,queens)
                    if (len(sidewaysMoves)>0):
                        (qtest,besti,bestj)=sidewaysMoves[random.randint(0,len(sidewaysMoves)-1)]
                        #print("Sideways Move"+str(sidewaysMovesCount))
                        if(qtest!=q):
                            print("OH GOD")
                    
                
            if (sidewaysMovesCount>=maxSidewaysMoves): # too many sideways moves
                print("too many sideways")
                return (None, steps)
            if (len(priorityQueensSideways)==0 and besti==None): # no sideways moves found, somehow
                print("no sideways")
                return (None, steps)
            queens[q]=(besti,bestj)
            sidewaysMovesCount=sidewaysMovesCount+1
            steps=steps+1
            
            

            
# hill climb search but it supports sideways moves up to a certain count in a row.
def althillClimbSearchSideways(n, maxSidewaysMoves, queens=None):
    steps=0
    if queens==None:
        queens = makeQueens(n)
    visited={str(queens):True} # store initial state of queens in a dictionary that will also store visited states
    #print(visited)
    sidewaysMovesCount=0
    while (True):
        goodMoveMade=False
        array = makeArray(n) 
        populateAttacking(array, queens) # populating the array with heuristic values
        priorityQueens = []
        heapq.heapify(priorityQueens) # array to store indexes of queens as sorted list by which one is worst
        q=0
        while(q<len(queens)): # sort queens by how many queens are attacking them
            (qi,qj)=queens[q]
            qH = array[qi][qj]-1
            heapq.heappush(priorityQueens, (qH*-1,q)) # heaps sort from smallest to largest and we want largest to smallest, so multiply by -1
            q=q+1
        (worstH,worst)=priorityQueens[0]
        priorityQueensSideways=priorityQueens.copy()
        if(worstH==0): # the worst queen is 0, we solved it
            return (queens, steps)
        qi=None
        qj=None
        qH=None
        (bestH,besti,bestj)=(None,None,None)
        while(len(priorityQueens)>0 and besti==None): # we want to check all the queens, stopping when we have an answer
            (qHNeg,q)=heapq.heappop(priorityQueens)
            qH=qHNeg*-1 # reverse the negativeness that we applied earlier
            (qi,qj)=queens[q] # position of the queen we are checking
            (bestH,besti,bestj)=bestMove(array,qi,qj,queens)
        if (besti!=None): # a best move was found, proceed.
            queens[q]=(besti,bestj)
            #print("good move "+str(bestH))
            goodMoveMade=True
            sidewaysMovesCount=0
            steps=steps+1
        if (goodMoveMade==False):
            # by virtue of getting here, we have reached the sideways moves.
            if(sidewaysMovesCount<maxSidewaysMoves):
                qi=None
                qj=None
                qH=None
                (besti,bestj)=(None,None)
                sidewaysMoves=[]
                while(len(priorityQueensSideways)>0): # we want to check all the queens, stopping when we have an answer
                    (qHNeg,q)=heapq.heappop(priorityQueensSideways)
                    qH=qHNeg*-1 # reverse the negativeness that we applied earlier
                    (qi,qj)=queens[q] # position of the queen we are checking
                    sidewaysMoves=sidewaysMoves+sidewaysFinder(array,qi,qj,queens)
                i=0
                #print("Sideways Moves: "+str(sidewaysMoves))
                #print(visited)
                while(i<len(sidewaysMoves)): # testing if a potential sideways move is in the visited dictionary
                    #print(queens)
                    (qtest,testi,testj)=sidewaysMoves[i]
                    #print(sidewaysMoves[i])
                    #print("qtest: "+str(qtest)+" testi: "+str(testi)+" testj: "+str(testj))
                    testqueens=queens.copy()
                    testqueens[qtest]=(testi,testj)
                    #print(visited)
                    if str(testqueens) in visited:
                        #print(queens)
                        #print(testqueens)
                        #print("duplicate")
                        sidewaysMoves.pop(i)
                    else:
                        #print("nonduplicate")
                        i=i+1
                    
                if (len(sidewaysMoves)>0):
                    (q,besti,bestj)=sidewaysMoves[random.randint(0,len(sidewaysMoves)-1)]
                    #print("Sideways Move"+str(sidewaysMovesCount))
                    
                
            if (sidewaysMovesCount>=maxSidewaysMoves): # too many sideways moves
                print("too many sideways")
                return (None, steps)
            if (len(sidewaysMoves)==0): # no sideways moves found, somehow
                print("no sideways")
                return (None, steps)
            queens[q]=(besti,bestj)
            sidewaysMovesCount=sidewaysMovesCount+1
            steps=steps+1
        visited[str(queens)]=True # store state of queens after move




In [303]:
array=makeArray(10)
queens=makeQueens(10)
showQueens(10, queens)
populateAttacking(array, queens)
printArray(array)
print(isAttacking((1,1),2,2))
print(isAttacking((1,1),3,2))
(qi,qj) = queens[3]
print(bestMove(array, qi, qj, queens))

0 0 0 0 0 0 0 0 Q 0 
0 0 0 Q 0 0 0 0 0 0 
0 0 0 0 0 Q 0 0 0 0 
0 0 0 0 0 0 Q 0 0 0 
0 0 0 0 0 0 Q 0 0 0 
0 0 0 0 0 0 0 0 0 Q 
0 0 0 0 0 Q 0 0 0 0 
0 0 0 0 0 0 Q 0 0 0 
0 Q 0 0 0 0 0 0 0 0 
0 0 Q 0 0 0 0 0 0 0 
1 2 4 4 3 3 4 2 1 4 
3 2 2 2 3 4 5 2 4 4 
1 4 3 2 3 3 6 3 3 4 
1 3 4 2 2 6 5 3 4 2 
2 2 2 5 2 5 4 5 3 3 
1 2 3 3 5 4 6 3 5 1 
1 3 3 4 2 4 4 2 5 4 
4 3 4 3 3 3 4 2 2 4 
2 3 3 4 1 4 5 3 2 2 
3 3 3 2 2 4 4 1 4 2 
1
0
(0, 3, 0)


In [565]:
n=8
queens=None
tries=0
steps=0
while(queens==None):
    (queens,steps)=hillClimbSearch(n)
    tries=tries+1
showQueens(n,queens)
print("Tries: "+str(tries))
print("Steps: "+str(steps))
queens=None
tries=0
steps=0
while(queens==None):
    (queens,steps)=althillClimbSearchSideways(n,100)
    tries=tries+1
showQueens(n, queens)
print("Tries: "+str(tries))
print("Steps: "+str(steps))

fails1=0
fails2=0
for i in range(0,100):
    (queens,steps)=hillClimbSearch(n)
    if(queens==None):
        fails1=fails1+1
    (queens,steps)=althillClimbSearchSideways(n,100)
    if(queens==None):
        fails2=fails2+1
print("regular fails: "+str(fails1))
print("sideways fails: "+str(fails2))

0 0 0 0 0 0 0 Q 
0 0 0 Q 0 0 0 0 
Q 0 0 0 0 0 0 0 
0 0 Q 0 0 0 0 0 
0 0 0 0 0 Q 0 0 
0 Q 0 0 0 0 0 0 
0 0 0 0 0 0 Q 0 
0 0 0 0 Q 0 0 0 
Tries: 1
Steps: 3
0 0 0 0 0 Q 0 0 
0 0 0 Q 0 0 0 0 
0 Q 0 0 0 0 0 0 
0 0 0 0 0 0 0 Q 
0 0 0 0 Q 0 0 0 
0 0 0 0 0 0 Q 0 
Q 0 0 0 0 0 0 0 
0 0 Q 0 0 0 0 0 
Tries: 1
Steps: 8
no sideways
no sideways
regular fails: 84
sideways fails: 2


        worst=None # index of worst queen in queens array
        worstH=0 # Heuristic value of the worst queen
        q=0
        while q<len(queens): # search for the worst queen
            (qi,qj)=queens[q]
            if array[qi][qj]-1>worstH:
                worst=q
                worstH=array[qi][qj]-1
            q=q+1
        if worst==None: # hey! we don't have a worst queen! We solved the puzzle!
            return queens
        (qi,qj)=queens[worst] # position of our worst queen
        (bestH,besti,bestj)=bestMove(array,qi,qj,queens)

In [561]:
array=makeArray(4)
queens=[(1,1),(2,3)]
populateAttacking(array,queens)
showQueens(4,queens)
printArray(array)
sidewaysFinder(array,1,1,queens)

0 0 0 0 
0 Q 0 0 
0 0 0 Q 
0 0 0 0 
1 2 1 1 
1 1 2 2 
2 2 2 1 
0 1 1 2 


[(0, 0, 0), (0, 0, 2), (0, 1, 0), (0, 3, 0), (0, 3, 1)]

In [562]:
althillClimbSearchSideways(n,100)

[(0, 4), (1, 7), (0, 2), (4, 1), (2, 5), (5, 6), (6, 0), (7, 3)]
(0, 3, 1)
qtest: 0 testi: 3 testj: 1
nonduplicate
[(0, 4), (1, 7), (0, 2), (4, 1), (2, 5), (5, 6), (6, 0), (7, 3)]
(0, 5, 4)
qtest: 0 testi: 5 testj: 4
nonduplicate
[(0, 4), (1, 7), (0, 2), (4, 1), (2, 5), (5, 6), (6, 0), (7, 3)]
(2, 3, 2)
qtest: 2 testi: 3 testj: 2
nonduplicate
[(0, 4), (1, 7), (0, 2), (4, 1), (2, 5), (5, 6), (6, 0), (7, 3)]
(2, 7, 2)
qtest: 2 testi: 7 testj: 2
nonduplicate
[(0, 4), (1, 7), (7, 2), (4, 1), (2, 5), (5, 6), (6, 0), (7, 3)]
(2, 0, 2)
qtest: 2 testi: 0 testj: 2
[(0, 4), (1, 7), (7, 2), (4, 1), (2, 5), (5, 6), (6, 0), (7, 3)]
[(0, 4), (1, 7), (0, 2), (4, 1), (2, 5), (5, 6), (6, 0), (7, 3)]
duplicate
[(0, 4), (1, 7), (7, 2), (4, 1), (2, 5), (5, 6), (6, 0), (7, 3)]
(2, 3, 2)
qtest: 2 testi: 3 testj: 2
nonduplicate
[(0, 4), (1, 7), (7, 2), (4, 1), (2, 5), (5, 6), (6, 0), (7, 3)]
(7, 3, 3)
qtest: 7 testi: 3 testj: 3
nonduplicate
[(0, 4), (1, 7), (3, 2), (4, 1), (2, 5), (5, 6), (6, 0), (7, 3)]
(2,

([(0, 3), (5, 7), (3, 2), (1, 1), (2, 6), (4, 5), (6, 0), (7, 4)], 13)